In [1]:
# Import Python package
import binhopulsar
from binhopulsar.pulsar import Pulsar
from binhopulsar.commands.system.definitions import GetUsbStringSubCommand

## Getting started

#### 1. List all the Pulsar devices connected to the PC host

The ``binhopulsar.getConnectedPulsarDevicesList()`` gets a list of the Pulsar devices plugged into the host PC machine.

In [2]:
binhopulsar.getConnectedPulsarDevicesList()

[{'path': '\\\\?\\HID#VID_1FC9&PID_82FD#7&271efd59&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}',
  'vendor_id': '0x1fc9',
  'product_id': '0x82fd',
  'serial_number': '0AA4AA0BBF431055B02059D06095F7BB',
  'manufacturer_string': 'Binho LLC',
  'product_string': 'Binho Pulsar',
  'hardware_version': 'B',
  'firmware_version': '4.1.2'}]

#### 2. Create an instance of the Pulsar class

To utilize a Pulsar USB host adapter device, we need to create an instance of the Pulsar class.

In [3]:
# Create a device instance. One instance of the Pulsar class represents a Pulsar USB host adapter device.
pulsar = Pulsar()

#### 3. Open connection to the Pulsar device

The public method ``Pulsar.open()`` establishes the connection with a Pulsar device. Below is the complete signature:

```python
open(serial, path)
```

- ``serial: str``: The Pulsar serial number.
- ``path: str``: The OS HID path assigned to the device. This can be obtained using the ``binhopulsar.getConnectedPulsarDevicesList()`` method. The ``path`` parameter is currently the only way to uniquely identify each Pulsar device. Therefore, it is recommended to use the ``path`` parameter, especially when opening connections with more than one Pulsar device simultaneously.

In [4]:
# Use the method by default to connect to only one Pulsar device.
pulsar.open()

# Otherwise, use the path attribute to identify each Pulsar device. Uncomment the line below and comment the line #2.
# pulsar.open(path='\\\\?\\HID#VID_1FC9&PID_82FC#6&48d9417&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}')

{'module': 0,
 'opcode': 0,
 'message': 'Connection with Pulsar device opened successfully.'}

#### 4. Define and register a callback to handle responses and notifications from Pulsar

To handle responses and notifications from Pulsar, a callback function must be defined and registered. This function will be invoked every time the Pulsar sends a response to a request, an asynchronous notification, or a message from the system.

The callback function's signature is as follows: 

``def callback_function_name(pulsar_message: dict, system_message: dict) -> None:``

Once the callback function is defined, it should be registered using the ``Pulsar.onEvent(callback_function)`` method.

In [5]:
# Define callback function
def callback_function(pulsar_message: dict, system_message: dict) -> None:

    if pulsar_message != None:

        # Print mesasage
        print(">> New message from PULSAR:")
        print(pulsar_message)

    if system_message != None:

        # Print message
        print(">> New message from the SYSTEM:")
        print(system_message)

In [6]:
# Register callback function
pulsar.onEvent(callback_function)

{'module': 0,
 'opcode': 0,
 'message': 'On event callback function registered successfully.'}

#### 5. Define a function to generate transaction IDs

All the request messages sent to the Pulsar from the USB Host application must include the transaction or request ID. The ID is a 2-byte integer with an allowed range of ``[1, 65535]``.

In this example, a dummy function called ``getId()`` is defined to increment a transaction counter used as the ID.

In [7]:
#Auxiliar code to generate IDs.

counter_id = 0

def getId():
    global counter_id
    counter_id = counter_id + 1
    return counter_id

## System API

```Pulsar.getDeviceInfo(id)```

This method retrieves the device metadata including: manufacturer, product name, serial number, hardware version, firmware version and the device supported groups.
This method intends to replace the `getUsbString` method.
- ``id: int`` : A 2-byte integer that represents the transfer ID.

In [8]:
pulsar.getDeviceInfo(getId())

{'module': 1, 'opcode': 0, 'message': 'GET DEVICE INFO request success'}

>> New message from PULSAR:
{'id': 1, 'command': 'SYS GET DEVICE INFO', 'result': 'SUCCESS', 'manufacturer': 'Binho LLC', 'product_name': 'Pulsar', 'serial_number': '0AA4AA0BBF431055B02059D06095F7BB', 'hardware_version': 'B', 'firmware_version': '4.1.2', 'capabilities': {'supported_groups': ['I2C', 'SPI', 'UART', 'GPIO']}}


``Pulsar.getUsbString(id, subCommand)``

The ``GET USB STRING`` command allows you to retrieve various string values related to the product, such as the manufacturer, product name, serial number, firmware version, and hardware version. This command serves as a basic test to verify the communication and functionality with the Pulsar device. By successfully receiving the expected string values, you can ensure that the communication between the PC host and the Pulsar device is functioning as expected.

- ``id: int`` : A 2-byte integer that represents the transfer ID.
- ``subCommand: GetUsbStringSubCommand``: The subcommand parameter indicates which string must be returned by the device.

In [9]:
# GET USB STRING - Get Manufacturer
request_result = pulsar.getUsbString(getId(), GetUsbStringSubCommand.MANUFACTURER)

# GET USB STRING  - Get Product Name
request_result = pulsar.getUsbString(getId(), GetUsbStringSubCommand.PRODUCT_NAME)

# GET USB STRING  - Get Serial Number
request_result = pulsar.getUsbString(getId(), GetUsbStringSubCommand.SERIAL_NUMBER)

# GET USB STRING  - Firmware Version
request_result = pulsar.getUsbString(getId(), GetUsbStringSubCommand.FW_VERSION)

# GET USB STRING  - Hardware Version
request_result = pulsar.getUsbString(getId(), GetUsbStringSubCommand.HW_VERSION)

>> New message from PULSAR:
{'id': 2, 'command': 'SYS GET USB STRING', 'result': 'SUCCESS', 'payload_length': 12, 'payload': 'MN-Binho LLC'}
>> New message from PULSAR:
{'id': 3, 'command': 'SYS GET USB STRING', 'result': 'SUCCESS', 'payload_length': 15, 'payload': 'PR-Binho Pulsar'}
>> New message from PULSAR:
{'id': 4, 'command': 'SYS GET USB STRING', 'result': 'SUCCESS', 'payload_length': 35, 'payload': 'SN-0AA4AA0BBF431055B02059D06095F7BB'}
>> New message from PULSAR:
{'id': 5, 'command': 'SYS GET USB STRING', 'result': 'SUCCESS', 'payload_length': 8, 'payload': 'FW-4.1.2'}
>> New message from PULSAR:
{'id': 6, 'command': 'SYS GET USB STRING', 'result': 'SUCCESS', 'payload_length': 4, 'payload': 'HW-B'}


``Pulsar.setI2cSpiUartGpioVoltage(id, voltage_mV)``

This method supplies the indicated voltage to the bus in mV, ranging from 1200 mV up to 3300 mV. It also allows setting the value 0 mV to power off the output voltage.

- ``id: int`` : A 2-byte integer that represents the transfer ID.
- ``voltage_mV: c_int16 ``: The voltage parameter is a 2-byte integer in the range [1200, 3300] mV and allows the value 0 mV to power off the output voltage.

In [10]:
pulsar.setI2cSpiUartGpioVoltage(getId(), 3300)

{'module': 1,
 'opcode': 0,
 'message': 'SET I2C SPI UART GPIO VOLTAGE request success'}

>> New message from PULSAR:
{'id': 7, 'command': 'SYS SET I2C SPI UART GPIO VOLTAGE', 'result': 'SUCCESS'}


``Pulsar.getAnalogMeasurements(id)``

This method returns the voltage of all the internal and external power domains, in mV.

- ``id: int`` : A 2-byte integer that represents the transfer ID.

In [11]:
pulsar.getAnalogMeasurements(getId())

{'module': 1,
 'opcode': 0,
 'message': 'GET ANALOG MEASUREMENTS request success'}

>> New message from PULSAR:
{'id': 8, 'command': 'SYS GET ANALOG MEASUREMENTS', 'result': 'SUCCESS', 'i2c_spi_uart_gpio_vtarg': {'internal_mV': 3295, 'external_mV': 3295}, 'i3c_low_voltage_vtarg': {'internal_mV': 0, 'external_mV': 0}, 'i3c_high_voltage_vtarg': {'internal_mV': 0, 'external_mV': 0}}


``Pulsar.useExternalI2cSpiUartGpioVoltage(id)``

This method indicates that an external voltage source is used to supply the I2C, SPI and UART buses as well as GPIOs interface, and the Pulsar must set its internal voltage to meet the external voltage. This methods returns the voltage measured in the VTARG pin.

- ``id: int`` : A 2-byte integer that represents the transfer ID.

In [12]:
pulsar.useExternalI2cSpiUartGpioVoltage(getId())

{'module': 1,
 'opcode': 0,
 'message': 'USE EXT SRC I2C-SPI-UART-GPIO VOLTAGE request success'}

>> New message from PULSAR:
{'id': 9, 'command': 'SYS USE EXTERNAL I2C SPI UART GPIO VOLTAGE', 'result': 'VOLTAGE_OUT_OF_RANGE', 'voltage_mV': 2}


``Pulsar.resetDevice(id)``

This method resets the Pulsar. As a result an `UsbDisconnectionError Exception` will be raised when the devices is reset. 

- ``id: int`` : A 2-byte integer that represents the transfer ID.


In [13]:
pulsar.resetDevice(getId())

>> New message from the SYSTEM:
{'module': 0, 'opcode': 5, 'message': 'Error UNEXPECTED_DISCONNECTION: Unexpected Pulsar disconnection.'}


{'module': 1, 'opcode': 0, 'message': 'RESET DEVICE request success'}

``Pulsar.enterBootMode(id)``

This method makes the Pulsar jump to the USB bootloader. As a result an `UsbDisconnectionError Exception` will be raised when the devices is reset and then a new USB mass storage device should be enumerated by the OS whose name should be **SUPERNOVA**. 

- ``id: int`` : A 2-byte integer that represents the transfer ID.


In [ ]:
pulsar.enterBootMode(getId())

## Close communication

Use the ``Pulsar.close()`` method to end the communication with the Pulsar device and release the used memory in the background like threads and so on.

In [ ]:
# Close the communication with the Pulsar device.
pulsar.close()